# Homework 2
#### Gavin Martin 
#### March 12, 2020
#### CS 344 

In [1]:
# define global corpuses for spam filtering 
good_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]
spam_corpus = [["i", "am", "spam", "spam", "i", "am"], ["i", "do", "not", "like", "that", "spamiam"]]

# this creates dictionary of word and its frequency in a corpus
def create_word_occurrence_dict(corpus):
    new_dict = {}
    for message in corpus:
        for word in message:
            if word in new_dict:
                #increment frequency
                new_dict[word] += 1
            else:
                new_dict.update({word: 1})
    return new_dict


# get word spam probability of individual word given good and bad corpus
def get_word_spam_probability(word, good_occurrence_dict, spam_occurrence_dict):
    good_freq = 0
    bad_freq = 0
    # get num occurences
    if word in good_occurrence_dict:
        good_freq = good_occurrence_dict[word] * 2
    if word in spam_occurrence_dict:
        bad_freq = spam_occurrence_dict[word]
    # get num messages in corpus
    num_good_messages = len(good_corpus)
    num_bad_messages = len(spam_corpus)
    # if it occurs more than 1x
    if good_freq + bad_freq > 1:
        # get probability
        word_value = max(.01, min(.99, min(1.0, bad_freq / num_bad_messages) / (
                min(1.0, good_freq/num_good_messages) + min(1.0, bad_freq/num_bad_messages))))
    else:
        # if not found, it has probability of .4
        word_value = .4
    return word_value


# get probability dictionary of all words in the message
def get_word_probability_dict(message):
    # Analyze each word from message
    global good_corpus
    global spam_corpus
    # get word frequency dictionaries from both corpus
    good_occurrence_dict = create_word_occurrence_dict(good_corpus)
    spam_occurrence_dict = create_word_occurrence_dict(spam_corpus)
    # split up message
    word_list = message.split()
    new_dict = {}
    for word in word_list:
        # get the probabilities and add them to dictionary
        probability = get_word_spam_probability(word, good_occurrence_dict, spam_occurrence_dict)
        new_dict.update({word: probability})
    return new_dict


# This gets the most interesting words (ones with values furthest from .5 and creates a dictionary with them
def get_most_interesting_words(word_dict):
    new_dict = {}
    max_length = 15
    # if the message is shorter than 15 words, that becomes max length
    if len(word_dict) < 15:
        max_length = len(word_dict)
    for x in range(max_length):
        # get the word probabilities
        probabilities = list(word_dict.values())
        # get the words themselves
        words = list(word_dict.keys())
        # get the max word from the current dictionary
        max_word = words[probabilities.index(max(probabilities))]
        # add it to new dictionary of important words
        new_dict.update({max_word: word_dict[max_word]})
        # delete previous max word so you can get next max
        word_dict.pop(max_word)
    return new_dict

# This gets the combined probability of a dictionary of word / probability pairs
def get_combined_probability(word_dict):
    product = 1
    inverse_product = 1
    for word in word_dict:
        product *= word_dict[word]
        inverse_product *= (1 - word_dict[word])
    return product / (product + inverse_product)


# Determines probability of spam message
def determine_spam_probability(message):
    # Gets the probability that each token is spam
    token_spam_probability_dict = get_word_probability_dict(message)
    # Gets the most interesting tokens from the message
    interesting_tokens_dict = get_most_interesting_words(token_spam_probability_dict)
    print(interesting_tokens_dict) # To show this works as intended
    # this gets combined probability of every word in message
    message_spam_probability = get_combined_probability(interesting_tokens_dict)
    # this rounds the decimal to the 4th place
    return round(message_spam_probability, 4)


test1_spam = "i am samiam and i do not like that spam"
test2_good = "i do i do like green eggs and ham "
test3_mixed = "i like green eggs and ham but only if it is not made by that spamiam because i do not like spam no spam no spam"
print("\n")
test1_probability = determine_spam_probability(test1_spam)
print("Test 1 Spam probability is " + str(test1_probability) + "\n")
test2_probability = determine_spam_probability(test2_good)
print("Test 2 Spam probability is " + str(test2_probability) + "\n")
test3_probability = determine_spam_probability(test3_mixed)
print("Test 3 Spam probability is " + str(test3_probability) + "\n")





{'am': 0.99, 'spam': 0.99, 'i': 0.5, 'samiam': 0.4, 'not': 0.4, 'that': 0.4, 'do': 0.3333333333333333, 'like': 0.3333333333333333, 'and': 0.01}
Test 1 Spam probability is 0.88

{'i': 0.5, 'do': 0.3333333333333333, 'like': 0.3333333333333333, 'green': 0.01, 'eggs': 0.01, 'and': 0.01, 'ham': 0.01}
Test 2 Spam probability is 0.0

{'spam': 0.99, 'i': 0.5, 'but': 0.4, 'only': 0.4, 'if': 0.4, 'it': 0.4, 'is': 0.4, 'not': 0.4, 'made': 0.4, 'by': 0.4, 'that': 0.4, 'spamiam': 0.4, 'because': 0.4, 'no': 0.4, 'like': 0.3333333333333333}
Test 3 Spam probability is 0.2762



Graham argues that this is a Baysian approach to SPAM. What makes it Bayesian?

- It is a Bayesian approach because we are taking into account the Bayesian network that is created through the corpuses which are hashed into probability distrubtions. We are taking the Spam and Not Spam corpus and making a large distrubtion table through the hashes which we then use to calculate the joint probability of each word in the phrases. We are effectivly calculating a cause and effect where the probability of this spam email is calculated using the effect of previous spam and not spam emails.

In [6]:
from probability import BayesNet, enumeration_ask, elimination_ask, gibbs_ask

# Utility variables
T, F = True, False

# From AIMA code (probability.py) - Fig. 14.2 - burglary example
cloudy = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: .1, F: .5}),
    ('Rain', 'Cloudy', {T: .8, F: .2}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.9, (F, T): 0.9, (F, F): 0.0}),
    ])


Compute the number of independent values in the full joint probability distribution for this domain. Assume that no conditional independence relations are known to hold between these values.

There are 4 nodes / variables making it 2 * 2 * 2 * 2 or 16 outcomes

Compute the number of independent values in the Bayesian network for this domain. Assume the conditional independence relations implied by the Bayes network.

Cloudy affects all of them and that has 2 entries

In [7]:
print("\nP(Cloudy)")
print(enumeration_ask('Cloudy', dict(), cloudy).show_approx())

print("\nP(Sprinkler | Cloudy)")
print(enumeration_ask('Sprinkler', dict(Cloudy=True), cloudy).show_approx())

print("\nP(Cloudy | Sprinkler and no Rain)")
print(enumeration_ask('Cloudy', dict(Sprinkler=True, Rain=False), cloudy).show_approx())

print("\nP(Wet Grass | sprinkler and cloudy and raining)")
print(enumeration_ask("WetGrass", dict(Sprinkler=True, Rain=True, Cloudy=True), cloudy).show_approx())

print("\nP(Cloudy | no wet grass)")
print(enumeration_ask("Cloudy", dict(WetGrass=False), cloudy).show_approx())



P(Cloudy)
False: 0.5, True: 0.5

P(Sprinkler | Cloudy)
False: 0.9, True: 0.1

P(Cloudy | Sprinkler and no Rain)
False: 0.952, True: 0.0476

P(Wet Grass | sprinkler and cloudy and raining)
False: 0.01, True: 0.99

P(Cloudy | no wet grass)
False: 0.639, True: 0.361


P(Cloudy)
i. This has .5 and it is independent, so it is .5 for both true and false

<.5, .5>

P(Sprinker | cloudy)

ii. 

<P(S|C), P(not S|C)>

<.1, 1-.1> or 

<.1 (true), .9 (false)>


P(Cloudy| the sprinkler is running and it’s not raining)
iii.  
<P(C)*P(S|C)*P(not R|C), P(not C)*P(S|not C)*P(not R|not C)>

<(.5)*(.1)*(1-.8), (1-.5)*(.5)*(1-.2)>

<.010, .2> 

<.0476 (true), .952 (false)>


P(WetGrass | it’s cloudy, the sprinkler is running and it’s raining)
iv.   

<P(W|s,r) * P(s|c) * P(r|c) * P(c), P(not W|s,r)*P(s|c)*P(c)>

<(.99)*(.1)*(.2)*(.5), (1-.99)*(.1)*(.8)*(.5)>

<.0396, .0004>

<.99 (true), .01 (false)>
                


P(Cloudy | not Wet grass)

<P(C) * P(not W|s,r)*P(s|c)*P(r|c), P(not C) * P(s|not c) * P(r|not c)>
<.639 (false), .361 (true)>